In [1]:
# ================================
# Standard Library
# ================================
import glob
import os
import re
from typing import Any, Dict, List, Optional

# ================================
# Third-Party Libraries
# ================================
import bitsandbytes
import numpy as np
import pandas as pd
import torch
import yaml
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from datasets import Dataset

# HuggingFace Transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback,
    GenerationConfig,
    Trainer,
    TrainingArguments,
    __version__ as HF_VER,
)

# PEFT (LoRA / QLoRA)
from peft import (
    LoraConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training,
)

# TRL (Supervised Fine-Tuning Trainer)
from trl import SFTConfig, SFTTrainer


In [2]:
def load_config(yaml_path):
    with open(yaml_path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)
    
file_paths = load_config(yaml_path="P3-config.yaml")
PARAMS = load_config(yaml_path="validation_params.yaml")
prompts = load_config(yaml_path="prompts.yaml")

In [3]:
val_df_path = file_paths["train_val_paths"]["val_path"]
val_df = pd.read_csv(val_df_path)
columns_to_drop = ['report', 'report_key']
val_df = val_df.drop(columns=columns_to_drop) 
val_df.head()

,Unnamed: 0,Drug,Receptor,PDB_ID
0,46,Leucovorin Calcium,Dihydrofolate reductase,"8WGN,7XI7,6DAV,5SDB,5SDA,5SD9,5SD8,5SD7,5SD6,4..."
1,16,Letrozole,Aromatase,"5JL9,5JL7,5JL6,5JKW,5JKV,4GL7,4GL5,3S7S,3S79,3..."
2,4,Adenosine triphosphate,Tyrosine-protein kinase ABL1,"5MO4,1OPK,1OPL,2FO0,8SSN,4XEY,6XR7,6XR6,2E2B,4..."
3,9,Dapagliflozin,Sodium/glucose cotransporter 2,"7YNK,7YNJ,7VSI,8HIN,8HG7,8HEZ,8HDH,8HB0"
4,28,Amphetamine,Trace amine-associated receptor-1,"8JSO,8JLR,8JLQ,8JLP,8JLO,8JLN,8UHB,8ZSS,8ZSP,8..."


In [4]:
def _dtype_from_str(s: str):
    s = (s or "").lower()
    if s in ("bf16", "bfloat16"):
        return torch.bfloat16
    if s in ("fp16", "float16", "half"):
        return torch.float16
    if s in ("fp32", "float32"):
        return torch.float32
    return torch.float16


def make_bnb_config(infer_cfg: Dict[str, Any]) -> BitsAndBytesConfig:
    """
    Build BitsAndBytesConfig from validation_params.yaml's `quantization` block.
    """
    q = infer_cfg["quantization"]
    compute_dtype = q.get("compute_dtype", "auto")

    # allow "auto" in YAML, resolve here
    if compute_dtype == "auto":
        compute_dtype = "bf16" if torch.cuda.is_bf16_supported() else "fp16"

    return BitsAndBytesConfig(
        load_in_4bit=q["load_in_4bit"],
        bnb_4bit_quant_type=q["quant_type"],
        bnb_4bit_use_double_quant=q["double_quant"],
        bnb_4bit_compute_dtype=_dtype_from_str(compute_dtype),
    )


# -------------------------------
# Model loading
# -------------------------------

def load_model_and_tokenizer_for_inference(
    infer_cfg: Dict[str, Any]
):
    """
    Load base model + LoRA adapter for inference.

    Uses:
      - 4-bit quantization (preferred) OR
      - merged 16-bit model if `merge_16bit` is True.
    """
    base_id = infer_cfg["base_model_id"]
    adapter_dir = infer_cfg["adapter_dir"]
    device_map = infer_cfg["batch"]["device_map"]
    merge_16bit = infer_cfg["merge_16bit"]

    tokenizer = AutoTokenizer.from_pretrained(
        base_id,
        use_fast=True,
        trust_remote_code=True,
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    if merge_16bit:
        # 16-bit merged model
        dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
        model = AutoModelForCausalLM.from_pretrained(
            base_id,
            device_map=device_map,
            torch_dtype=dtype,
            trust_remote_code=True,
        )
        model = PeftModel.from_pretrained(model, adapter_dir)
        model = model.merge_and_unload()
    else:
        # 4-bit quantized path
        bnb_cfg = make_bnb_config(infer_cfg)
        model = AutoModelForCausalLM.from_pretrained(
            base_id,
            quantization_config=bnb_cfg,
            device_map=device_map,
            trust_remote_code=True,
        )
        model = PeftModel.from_pretrained(model, adapter_dir)

    model.eval()
    return model, tokenizer


# -------------------------------
# Prompt composition
# -------------------------------


PLACEHOLDER_RE = re.compile(r"\{([A-Za-z0-9_]+)\}")

def render_template_safe(template: str, values: Dict[str, Any]) -> str:
    """
    Replace {Key} placeholders using a regex, leaving all other braces intact.
    This prevents JSON examples like { "a": 1 } from being treated as format fields
    because we do NOT call .format().
    Only patterns that look like {WordLikeKey} are replaced.
    """
    def repl(m):
        key = m.group(1)
        v = values.get(key, "")
        return "" if v is None else str(v)
    return PLACEHOLDER_RE.sub(repl, template or "")


def compose_chat_prompt_llama32(
    system_prompt: str,
    user_template: str,
    values: Dict[str, Any],
    tokenizer,
    few_shots: Optional[List[Dict[str, str]]] = None,
) -> str:
    """
    Llama-3.2 Instruct chat formatting:
    - system message is its own role
    - user message contains rendered template text
    - apply_chat_template handles <|begin_of_text|> etc
    """
    user_text = render_template_safe(user_template, values)

    msgs: List[Dict[str, str]] = []
    if system_prompt and str(system_prompt).strip():
        msgs.append({"role": "system", "content": system_prompt})

    # Optional few-shot examples (same structure as your training config)
    for shot in few_shots or []:
        if shot and "user" in shot and "assistant" in shot:
            msgs.append({"role": "user", "content": shot["user"]})
            msgs.append({"role": "assistant", "content": shot["assistant"]})

    msgs.append({"role": "user", "content": user_text})

    # For inference, we want the assistant header/prefix to be appended
    return tokenizer.apply_chat_template(
        msgs,
        tokenize=False,
        add_generation_prompt=True,
    )


def compose_plain_prompt_safe(
    user_template: str,
    values: Dict[str, Any],
) -> str:
    """
    Plain prompt version that is also JSON-brace-safe.
    """
    return render_template_safe(user_template, values)

# -------------------------------
# Row-level generation
# -------------------------------

@torch.no_grad()
def generate_for_row(
    row: Dict[str, Any],
    model,
    tokenizer,
    infer_cfg: Dict[str, Any],
    prompts_cfg: Dict[str, Any],
    test: bool,
) -> str:
    col_cfg = infer_cfg["columns"]
    col_d = col_cfg["drug"]
    col_r = col_cfg["receptor"]
    col_p = col_cfg["pdb"]
    col_rep = col_cfg.get("report")  # OPTIONAL: add this in your validation_params.yaml if you have report

    drug = str(row.get(col_d, "") or "")
    receptor = str(row.get(col_r, "") or "")

    pdb_raw = str(row.get(col_p, "") or "")
    pdb_ids = ",".join([p.strip() for p in pdb_raw.split(",") if p.strip()]) if pdb_raw else ""

    report = ""
    if col_rep:
        report = str(row.get(col_rep, "") or "")

    p_cfg = prompts_cfg.get("prompts", prompts_cfg)
    use_chat_format = p_cfg.get("use_chat_format", False)
    system_prompt = p_cfg.get("system", "")
    few_shots = p_cfg.get("few_shots", [])

    # Pick the right template for validation vs test
    user_template = p_cfg["test_user_template"] if test else p_cfg["val_user_template"]

    values = {
        "Drug": drug,
        "Receptor": receptor,
        "PDB_ID": pdb_ids,
        "report": report,   # IMPORTANT: supports {report} in templates
    }

    if use_chat_format:
        prompt = compose_chat_prompt_llama32(
            system_prompt=system_prompt,
            user_template=user_template,
            values=values,
            tokenizer=tokenizer,
            few_shots=few_shots,
        )
    else:
        prompt = compose_plain_prompt_safe(
            user_template=user_template,
            values=values,
        )

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        add_special_tokens=False,
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    gen_cfg = infer_cfg["generation"]
    out = model.generate(
        **inputs,
        max_new_tokens=gen_cfg["max_new_tokens"],
        do_sample=gen_cfg["do_sample"],
        temperature=gen_cfg["temperature"],
        top_p=gen_cfg["top_p"],
        top_k=gen_cfg["top_k"],
        repetition_penalty=gen_cfg["repetition_penalty"],
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    generated = out[0][inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(generated, skip_special_tokens=True)

    for s in gen_cfg.get("stop_strings", []):
        if s and s in text:
            text = text.split(s)[0].rstrip()
            break

    return text.strip()

# -------------------------------
# Output path helper
# -------------------------------

def get_output_path(
    infer_cfg: Dict[str, Any],
    test: bool,
    relation: bool,
    real_world: bool,
) -> str:
    """
    Decide which CSV path to save to based on flags.
    Uses the `save_paths` block from validation_params.yaml.
    """
    paths = infer_cfg["save_paths"]

    if not test:
        return paths["inference_reports"]

    if real_world:
        return paths["real_world_test"]

    if relation:
        return paths["some_relation"]

    return paths["no_relation"]


# -------------------------------
# DataFrame-level inference
# -------------------------------

def run_inference(
    df: pd.DataFrame,
    model,
    tokenizer,
    infer_cfg: Dict[str, Any],
    prompts_cfg: Dict[str, Any],
    test: bool,
    relation: bool = False,
    real_world: bool = False,
) -> pd.DataFrame:
    """
    Run inference over all rows in a DataFrame.

    Args:
        df: Input examples (must contain columns specified in infer_cfg["columns"]).
        model, tokenizer: Loaded via `load_model_and_tokenizer_for_inference`.
        infer_cfg: validation_params.yaml contents.
        prompts_cfg: prompts.yaml contents.
        test: If False => validation-like set; if True => test scenarios.
        relation: If True (and test=True), saves to "some_relation" path.
        real_world: If True (and test=True), saves to "real_world_test" path.

    Returns:
        DataFrame with an extra column `generated_report`.
    """
    records = []
    for _, row in df.iterrows():
        report = generate_for_row(
            row=row.to_dict(),
            model=model,
            tokenizer=tokenizer,
            infer_cfg=infer_cfg,
            prompts_cfg=prompts_cfg,
            test=test,
        )
        rec = dict(row)
        rec["generated_report"] = report
        records.append(rec)

    out_df = pd.DataFrame(records)
    out_path = get_output_path(
        infer_cfg=infer_cfg,
        test=test,
        relation=relation,
        real_world=real_world,
    )
    out_df.to_csv(out_path, index=False)
    return out_df

In [5]:
model, tokenizer = load_model_and_tokenizer_for_inference(PARAMS)

val_results_df = run_inference(
    df=val_df,
    model=model,
    tokenizer=tokenizer,
    infer_cfg=PARAMS,
    prompts_cfg=prompts,
    test=False,
)